## Importación de librerías

In [73]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier, Pool
from catboost import cv
from sklearn.model_selection import StratifiedKFold, GridSearchCV, cross_val_score, cross_validate
from sklearn.metrics import precision_score, accuracy_score, make_scorer
from sklearn.preprocessing import StandardScaler




## Cargar archivos X_train y_train y X_test



In [123]:
# Leer el archivo CSV
X_train = pd.read_csv(r'C:\Users\Administrator\Documents\Proyecto Spaceship Titanic\spaceship_titanic_proyecto\Data\X_train.csv')
y_train = pd.read_csv(r'C:\Users\Administrator\Documents\Proyecto Spaceship Titanic\spaceship_titanic_proyecto\Data\y_train.csv')
X_test = pd.read_csv(r'C:\Users\Administrator\Documents\Proyecto Spaceship Titanic\spaceship_titanic_proyecto\Data\X_test.csv')

## Preprocesamiento para el Random Forest y Catboost

In [124]:
#Hacer el encoding para X_train con one-hot-encoding:

X_train_encoded = pd.get_dummies(X_train, columns=['HomePlanet', 'Destination', 'CryoSleep', 'VIP'], drop_first=True)

# Convertir PassengerId a entero en X_train
X_train_encoded['PassengerId'] = X_train_encoded['PassengerId'].astype(int)

# Verificar valores no numéricos en PassengerId
non_numeric = pd.to_numeric(X_train_encoded['PassengerId'], errors='coerce')

# Contar los valores NaN que se han generado
num_nan = non_numeric.isna().sum()

print(f"Número de valores no numéricos en PassengerId para X_train: {num_nan}")


#Hacer el encoding para y_train con label encoding



# Crear y ajustar el codificador
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)

# Crear un DataFrame con los valores originales y los codificados
mapping_df = pd.DataFrame({
    'Valor_Original': label_encoder.classes_,
    'Valor_Codificado': range(len(label_encoder.classes_))
})

# Verificar el mapeo
print(mapping_df)

# Ejemplo de cómo ver los resultados codificados
print("Valores codificados:", y_train_encoded)






#Hacer el encoding para X_test con one-hot-encoding:

X_test_encoded = pd.get_dummies(X_test, columns=['HomePlanet', 'Destination', 'CryoSleep', 'VIP'], drop_first=True)

# Convertir PassengerId a entero en X_test
X_test_encoded['PassengerId'] = X_test_encoded['PassengerId'].astype(int)


# Verificar valores no numéricos en PassengerId
non_numeric = pd.to_numeric(X_test_encoded['PassengerId'], errors='coerce')

# Contar los valores NaN que se han generado
num_nan = non_numeric.isna().sum()

print(f"Número de valores no numéricos en PassengerId para X_test: {num_nan}")



# Asegurarme de que ambos DataFrames tengan las mismas columnas
X_test_encoded = X_test_encoded.reindex(columns=X_train_encoded.columns, fill_value=0)





Número de valores no numéricos en PassengerId para X_train: 0
   Valor_Original  Valor_Codificado
0           False                 0
1            True                 1
Valores codificados: [0 1 0 ... 1 0 1]
Número de valores no numéricos en PassengerId para X_test: 0


c:\Users\Administrator\Documents\Proyecto Spaceship Titanic\myenv\Lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


# Estandarización de X_train_encoded

In [125]:

import pandas as pd
from sklearn.preprocessing import StandardScaler

# Crear un escalador
scaler = StandardScaler()

# Identificar las columnas a estandarizar (todas menos 'PassengerId')
columns_to_scale = X_train_encoded.columns.difference(['PassengerId'])

# Aplicar la estandarización usando el DataFrame correcto
X_train_encoded[columns_to_scale] = scaler.fit_transform(X_train_encoded[columns_to_scale])

# Verificar los resultados
print(X_train_encoded.head())



   PassengerId       Age  RoomService  FoodCourt  ShoppingMall       Spa  \
0          101  0.709437    -0.340590  -0.287314     -0.290817 -0.276663   
1          201 -0.336717    -0.175364  -0.281669     -0.248968  0.211505   
2          301  2.034566    -0.275409   1.955616     -0.290817  5.694289   
3          302  0.290975    -0.340590   0.517406      0.330225  2.683471   
4          401 -0.894666     0.118709  -0.243409     -0.038048  0.225732   

     VRDeck  HomePlanet_Europa  HomePlanet_Mars  Destination_PSO J318.5-22  \
0 -0.269023           1.754795        -0.503664                  -0.317487   
1 -0.230194          -0.569867        -0.503664                  -0.317487   
2 -0.225782           1.754795        -0.503664                  -0.317487   
3 -0.098708           1.754795        -0.503664                  -0.317487   
4 -0.267258          -0.569867        -0.503664                  -0.317487   

   Destination_TRAPPIST-1e  CryoSleep_True  VIP_True  
0                 0

## Estandarización de X_test_encoded

In [126]:
# Crear un escalador
scaler = StandardScaler()

# Identificar las columnas a estandarizar (todas menos 'PassengerId')
columns_to_scale = X_test_encoded.columns.difference(['PassengerId'])

# Aplicar la estandarización usando el DataFrame correcto
X_test_encoded[columns_to_scale] = scaler.fit_transform(X_test_encoded[columns_to_scale])

# Verificar los resultados
print(X_test_encoded.head())

   PassengerId       Age  RoomService  FoodCourt  ShoppingMall       Spa  \
0         1301 -0.118479    -0.364952  -0.291657     -0.319712 -0.274731   
1         1801 -0.688858    -0.364952  -0.285690     -0.319712  2.282833   
2         1901  0.166710    -0.364952  -0.291657     -0.319712 -0.274731   
3         2101  0.665792    -0.364952   4.118210     -0.319712 -0.110749   
4         2301 -0.617561    -0.348316  -0.291657      0.825893 -0.274731   

     VRDeck  HomePlanet_Europa  HomePlanet_Mars  Destination_PSO J318.5-22  \
0 -0.251472          -0.553131        -0.525314                  -0.315862   
1 -0.251472          -0.553131        -0.525314                  -0.315862   
2 -0.251472           1.807889        -0.525314                  -0.315862   
3  0.222162           1.807889        -0.525314                  -0.315862   
4 -0.251472          -0.553131        -0.525314                  -0.315862   

   Destination_TRAPPIST-1e  CryoSleep_True  VIP_True  
0                 0

## Validación cruzada de Random Forest

In [127]:


# Asegúrate de que PassengerId no esté en X_train_encoded
X_train_filtered = X_train_encoded.drop(columns=['PassengerId'])

# Inicializar el modelo Random Forest con los mejores hiperparámetros
rf_model = RandomForestClassifier(
    random_state=42,
    max_depth=10,
    min_samples_split=5,
    n_estimators=300
)

# Definir los puntajes de precisión y exactitud
scoring = {
    'accuracy': make_scorer(accuracy_score),
    'precision': make_scorer(precision_score)
}

# Realizar validación cruzada con múltiples métricas
scores = cross_validate(rf_model, X_train_filtered, y_train_encoded, cv=10, scoring=scoring)

# Mostrar los resultados
print(f"Puntajes de precisión en cada pliegue: {scores['test_precision']}")
print(f"Precisión promedio: {scores['test_precision'].mean()}")

print(f"Puntajes de exactitud en cada pliegue: {scores['test_accuracy']}")
print(f"Exactitud promedio: {scores['test_accuracy'].mean()}")




Puntajes de precisión en cada pliegue: [0.77254098 0.76382979 0.7653277  0.75664622 0.77111111 0.83004926
 0.83095238 0.79738562 0.78850103 0.76939203]
Precisión promedio: 0.7845736117469061
Puntajes de exactitud en cada pliegue: [0.80229885 0.7816092  0.78505747 0.78596087 0.77790564 0.80437284
 0.81588032 0.81012658 0.81933257 0.79171461]
Exactitud promedio: 0.7974258957977859


## Ajustar parámetros del Random Forest

In [129]:


# Supongamos que ya tienes X_train_encoded y y_train_encoded definidos

# Definir el modelo
rf_model = RandomForestClassifier(random_state=42)

# Definir los parámetros que deseas ajustar
parametros = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
}

# Configurar GridSearchCV
grid_search = GridSearchCV(estimator=rf_model, param_grid=parametros, cv=10, scoring='accuracy')

# Aplicar GridSearchCV sin eliminar PassengerId
grid_search.fit(X_train_encoded.drop(columns=['PassengerId']), y_train_encoded)

# Obtener el mejor modelo y los mejores hiperparámetros
mejor_modelo = grid_search.best_estimator_
print(f"Mejores hiperparámetros: {grid_search.best_params_}")


Mejores hiperparámetros: {'max_depth': 10, 'min_samples_split': 5, 'n_estimators': 300}


## Validación cruzada de Catboost

In [76]:


# Asegúrate de que PassengerId no esté en X_train_encoded
X_train_filtered = X_train_encoded.drop(columns=['PassengerId'])

# Inicializar el modelo CatBoost con los mejores hiperparámetros
cat_model = CatBoostClassifier(
    iterations=500,
    learning_rate=0.1,
    depth=8,
    l2_leaf_reg=3,         # Mejores hiperparámetros
    bagging_temperature=0,  # Mejores hiperparámetros
    loss_function='Logloss',
    verbose=0
)

# Configurar la validación cruzada
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

precisiones = []
exactitudes = []

for train_index, val_index in skf.split(X_train_filtered, y_train_encoded):
    X_train_fold, X_val_fold = X_train_filtered.iloc[train_index], X_train_filtered.iloc[val_index]
    y_train_fold, y_val_fold = y_train_encoded[train_index], y_train_encoded[val_index]

    # Ajustar el modelo en el pliegue de entrenamiento
    cat_model.fit(X_train_fold, y_train_fold, eval_set=(X_val_fold, y_val_fold), verbose=0)
    
    # Hacer predicciones en el pliegue de validación
    y_val_pred = cat_model.predict(X_val_fold)
    
    # Calcular precisión y exactitud
    precision = precision_score(y_val_fold, y_val_pred)
    accuracy = accuracy_score(y_val_fold, y_val_pred)
    
    precisiones.append(precision)
    exactitudes.append(accuracy)

# Calcular promedios
precision_promedio = sum(precisiones) / len(precisiones)
exactitud_promedio = sum(exactitudes) / len(exactitudes)

print(f"Precisión promedio: {precision_promedio}")
print(f"Exactitud promedio: {exactitud_promedio}")



Precisión promedio: 0.7760733364858174
Exactitud promedio: 0.7942012624479633


## Ajustar hiperparámetros del Catboost
GridSearchCV es una herramienta en scikit-learn que te ayuda a buscar de manera exhaustiva la mejor combinación de hiperparámetros para un modelo. Funciona probando todas las combinaciones posibles de hiperparámetros que le especifiques en un conjunto de validación cruzada. 

In [75]:


# Asegúrate de que PassengerId no esté en X_train_encoded
X_train_filtered = X_train_encoded.drop(columns=['PassengerId'])

# Configuración de los parámetros para la búsqueda en cuadrícula
param_grid = {
    'iterations': [500, 1000],
    'learning_rate': [0.01, 0.1, 0.2],
    'depth': [4, 6, 8],
    'l2_leaf_reg': [1, 3, 5],
    'bagging_temperature': [0, 1, 2]
}

# Inicializar el modelo CatBoost
cat_model = CatBoostClassifier(loss_function='Logloss', verbose=0)

# Usar StratifiedKFold para la validación cruzada
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Definir la búsqueda en cuadrícula
grid_search = GridSearchCV(estimator=cat_model,
                           param_grid=param_grid,
                           scoring=make_scorer(precision_score),
                           cv=skf,
                           n_jobs=-1,
                           verbose=1)

# Ajustar el modelo
grid_search.fit(X_train_filtered, y_train_encoded)

# Mostrar los mejores parámetros y la mejor precisión
print(f"Mejores parámetros: {grid_search.best_params_}")
print(f"Mejor precisión: {grid_search.best_score_}")



Fitting 5 folds for each of 162 candidates, totalling 810 fits


c:\Users\Administrator\Documents\Proyecto Spaceship Titanic\myenv\Lib\site-packages\numpy\ma\core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


Mejores parámetros: {'bagging_temperature': 0, 'depth': 8, 'iterations': 500, 'l2_leaf_reg': 3, 'learning_rate': 0.1}
Mejor precisión: 0.7824324381069443


## Visualizar el equilibrio de las clases para seleccionar el mejor modelo

In [71]:
class_counts = y_train.value_counts()
print(class_counts)


Transported
True           4378
False          4315
Name: count, dtype: int64


## Interpretación de las métricas
Si el objetivo es minimizar los falsos positivos, la precisión es crucial, y el Random Forest sería preferible.
Si buscas un buen equilibrio general entre clases y deseas maximizar la exactitud, el CatBoost podría ser más adecuado.

Selecciono el random forest

## Entrenamiento del random forest con los mejores parámetros

In [79]:
from sklearn.ensemble import RandomForestClassifier

# Asegúrate de que PassengerId no esté en X_train_encoded
X_train_filtered = X_train_encoded.drop(columns=['PassengerId'])

# Inicializar el modelo Random Forest con los mejores hiperparámetros
rf_model_final = RandomForestClassifier(
    random_state=42,
    max_depth=10,
    min_samples_split=5,
    n_estimators=300
)

# Ajustar el modelo a todos los datos de entrenamiento
rf_model_final.fit(X_train_filtered, y_train_encoded)

# Ahora rf_model_final es el mejor modelo entrenado con los mejores hiperparámetros



RandomForestClassifier(max_depth=10, min_samples_split=5, n_estimators=300,
                       random_state=42)

In [86]:
print(y_train_encoded)

[0 1 0 ... 1 0 1]


## Usar el modelo entrenado de Random Forest para predecir y_pred con X_test_encoded

In [131]:
import pandas as pd
import numpy as np

# Suponiendo que ya tienes X_test_encoded y rf_model_final definidos

# Convertir PassengerId a string y verificar
X_test_encoded['PassengerId'] = X_test_encoded['PassengerId'].astype(str)
print("Tipo de PassengerId después de conversión inicial:", X_test_encoded['PassengerId'].dtype)

# Realizar las predicciones
y_pred = rf_model_final.predict(X_test_encoded.drop(columns=['PassengerId']))





Tipo de PassengerId después de conversión inicial: object


## Crear el csv para enviar a Kaggle

In [133]:
import pandas as pd

# Cargar el conjunto de datos de prueba
test_df = pd.read_csv(r'C:\Users\Administrator\Documents\Proyecto Spaceship Titanic\spaceship_titanic_proyecto\Data\spaceship-titanic\test.csv')

# Cortar el DataFrame a las primeras 4277 filas
test_df_cortado = test_df.iloc[:4277]

# Convertir y_pred a DataFrame
y_pred_df = pd.DataFrame(y_pred, columns=['Transported'])  # No necesitas index=False aquí

# Cortar y_pred_df a 4277 filas
y_pred_cortado = y_pred_df.iloc[:4277]

# Crear el DataFrame de envío
submission = pd.DataFrame({
    'PassengerId': test_df_cortado['PassengerId'],
    'Transported': y_pred_cortado['Transported']  # Asegúrate de acceder a la columna correcta
})

# Convertir Transported a booleano
submission['Transported'] = submission['Transported'].astype(bool)

# Guardar el DataFrame en un CSV
submission.to_csv('submission10.csv', index=False)

# Verificar el contenido del DataFrame de envío
print("Primeras filas del DataFrame de envío:\n", submission.head())

# Verificar el tipo de la columna PassengerId
print("Tipo de PassengerId en el DataFrame de envío:", submission['PassengerId'].dtype)

# Comprobar si todos los valores en PassengerId son strings
all_str = submission['PassengerId'].apply(lambda x: isinstance(x, str)).all()
print("¿Todos los valores en 'PassengerId' son str?:", all_str)


Primeras filas del DataFrame de envío:
   PassengerId  Transported
0     0013_01         True
1     0018_01        False
2     0019_01         True
3     0021_01         True
4     0023_01         True
Tipo de PassengerId en el DataFrame de envío: object
¿Todos los valores en 'PassengerId' son str?: True


## Entrenar el modelo de Catboost

In [119]:
import pandas as pd
from catboost import CatBoostClassifier

# Supongamos que ya tienes X_train y y_train preparados
# Cargar los datos (ajusta esto a tus datos)
# X_train = ...
# y_train = ...

# Parámetros óptimos que has encontrado
params = {
    'iterations': 1000,       # Número de iteraciones
    'learning_rate': 0.1,     # Tasa de aprendizaje
    'depth': 6,               # Profundidad del árbol
    'l2_leaf_reg': 3,         # Regularización L2
    'eval_metric': 'AUC',     # Métrica de evaluación
    'random_seed': 42,        # Semilla aleatoria
    'verbose': 100            # Frecuencia de la salida de progreso
}

# Inicializar el modelo
catboost_model = CatBoostClassifier(**params)

# Entrenar el modelo
catboost_model.fit(X_train_encoded, y_train_encoded)

# Guardar el modelo si lo deseas
catboost_model.save_model('catboost_model.cbm')


0:	total: 7.41ms	remaining: 7.4s
100:	total: 567ms	remaining: 5.05s
200:	total: 1.13s	remaining: 4.48s
300:	total: 1.62s	remaining: 3.76s
400:	total: 2.11s	remaining: 3.15s
500:	total: 2.6s	remaining: 2.59s
600:	total: 3.09s	remaining: 2.05s
700:	total: 3.6s	remaining: 1.54s
800:	total: 4.1s	remaining: 1.02s
900:	total: 4.59s	remaining: 504ms
999:	total: 5.07s	remaining: 0us


## Probar el modelo de Catboost con X_test_encoded

In [121]:
# Asegúrate de que X_test_encoded esté preprocesado adecuadamente
# X_test_encoded = ...  # Tu DataFrame de prueba debe estar preparado

# Realizar las predicciones
y_pred = catboost_model.predict(X_test_encoded)

# Si deseas ver las predicciones
print("Primeras predicciones:\n", y_pred[:10])




CatBoostError: Bad value for num_feature[non_default_doc_idx=0,feature_idx=0]="Earth": Cannot convert 'b'Earth'' to float